# Install necessary libraries

In [ ]:
!pip3 install matplotlib pandas graphviz bs4 scikit-learn

# Import the libraries

In [4]:
import os
import re
import os
import tqdm
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import graphviz

from collections import defaultdict
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score

# HTML Parsing

Build the id to conversation dictionary

In [12]:
data_path = "data/html/*.html"

id_to_conversation = dict()

pbar = tqdm.tqdm(sorted(list(glob(data_path))))
for path in pbar:

    file_id = os.path.basename(path).split(".")[0]

    with open(path, "r", encoding="latin1") as file:
        html = file.read()
        soup = BeautifulSoup(html, "html.parser")

        conversations = soup.find_all("div", attrs={"data-testid": re.compile(r"conversation-turn-[0-9]+")})

        conversation_texts = []
        for i, conversation in enumerate(conversations):
            conversation = conversation.find_all("div", attrs={"data-message-author-role": re.compile(r"[user|assistant]")})
            
            if len(conversation) > 0:
                role = conversation[0].get("data-message-author-role")
                conversation_texts.append(
                    {
                        "role" : role,
                        "text" : conversation[0].text
                    }
                )
        
        id_to_conversation[file_id] = conversation_texts

100%|██████████| 127/127 [00:15<00:00,  8.17it/s]


Example conversation element (message)

In [19]:
pprint(id_to_conversation["0031c86e-81f4-4eef-9e0e-28037abf9883"][0])

{'role': 'user',
 'text': 'Load a CSV file into a Pandas in Python. The file is named '
         "'cs412_hw1_dataset.csv' and contains columns like 'Species', "
         "'Island', 'Sex', 'Diet', 'Year', 'Life Stage', 'Body Mass (g)', "
         "'Bill Length (mm)', 'Bill Depth (mm)', 'Flipper Length (mm)', and "
         "'Health Metrics'. \n"}


# Prompt Matching

Build the id to user prompts dictionary, and collect all user prompts in a list

In [21]:
all_user_prompts = []

id_to_user_prompts = dict()

for id, conversation in id_to_conversation.items():
    user_prompts = []

    for message in conversation:
        if message["role"] == "user":
            all_user_prompts.append(message["text"])
            user_prompts.append(message["text"])
    
    id_to_user_prompts[id] = user_prompts

In [24]:
# This list contains every prompt that students made
print(len(all_user_prompts))
print(all_user_prompts[0])

3544
Load a CSV file into a Pandas in Python. The file is named 'cs412_hw1_dataset.csv' and contains columns like 'Species', 'Island', 'Sex', 'Diet', 'Year', 'Life Stage', 'Body Mass (g)', 'Bill Length (mm)', 'Bill Depth (mm)', 'Flipper Length (mm)', and 'Health Metrics'. 

